# Lab 8

- Daniel Alfredo Rayo Rodan

- Gerardo Gabriel Pineda Riveiro

In [0]:
!pip install openpyxl

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import col, avg, substring, split
import pandas as pd

## Cargado de datos

Para la carga, se tuvo que especificar el tipo explicitamente de algunas variables para que spark fuera capaz de cargarlos.

In [0]:
pandas_df = pd.read_excel("/Volumes/workspace/default/lab8/hechos.xlsx", sheet_name="Sheet1")
pandas_df["hora_ocu"] = pd.to_numeric(pandas_df["hora_ocu"], errors="coerce")
pandas_df["zona_ocu"] = pandas_df["zona_ocu"].astype(str)
pandas_df["modelo_veh"] = pandas_df["modelo_veh"].astype(str)

spark = SparkSession.builder \
    .appName("ExcelReader") \
    .getOrCreate()

hechos_df = spark.createDataFrame(pandas_df)

pandas_df = pd.read_excel("/Volumes/workspace/default/lab8/fallecidos.xlsx", sheet_name="Sheet1")
pandas_df["hora_ocu"] = pd.to_numeric(pandas_df["hora_ocu"], errors="coerce")
pandas_df["edad_per"] = pd.to_numeric(pandas_df["edad_per"], errors="coerce")
pandas_df["zona_ocu"] = pandas_df["zona_ocu"].astype(str)
pandas_df["modelo_veh"] = pandas_df["modelo_veh"].astype(str)
fallecidos_df = spark.createDataFrame(pandas_df)
pandas_df = pd.read_excel("/Volumes/workspace/default/lab8/vehiculos.xlsx", sheet_name="Sheet1")
pandas_df["hora_ocu"] = pd.to_numeric(pandas_df["hora_ocu"], errors="coerce")
pandas_df["edad_per"] = pd.to_numeric(pandas_df["edad_per"], errors="coerce")
pandas_df["zona_ocu"] = pandas_df["zona_ocu"].astype(str)
pandas_df["modelo_veh"] = pandas_df["modelo_veh"].astype(str)
vehiculos_df = spark.createDataFrame(pandas_df)
pandas_df = 0

## Análisis exploratorio

Empezando con la forma de los datos estudiados, se esta trabajando con 3 datasets diferentes, sobre accidentes de tráfico ocurridos en guatemala registrados por la Policia Nacional Civil, a continuaión se puede ver la cantidad de registros por dataset:

In [0]:
print("REGISTROS DE FALLECIDOS:")
print(fallecidos_df.count())
print("REGISTROS DE HECHOS DE TRANSITO:")
print(hechos_df.count())
print("REGISTROS DE VEHICULOS:")
print(vehiculos_df.count())

REGISTROS DE FALLECIDOS:
11198
REGISTROS DE HECHOS DE TRANSITO:
8218
REGISTROS DE VEHICULOS:
12197


El dataset mas grande es de vehiculos, seguido de fallecidos y hechos de transito.

A continuación se muestran pequeñas muestras de los datos además de algunas características descriptivas de las variables de cada dataset. Se podrá observar que se presentan variables geograficas y temporales similares, que después se puede utilizar para relacionar los datos, dado que es poco probable que varios accidentes ocurrran en la misma hora, y lugar.

**Fallecidos**

In [0]:
display(fallecidos_df.limit(2))

núm_corre,año_ocu,día_ocu,hora_ocu,g_hora,g_hora_5,mes_ocu,día_sem_ocu,depto_ocu,mupio_ocu,zona_ocu,sexo_per,edad_per,g_edad_80ymás,g_edad_60ymás,edad_quinquenales,mayor_menor,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,tipo_eve,fall_les,int_o_noint
1,2023,1,15.0,12:00 a 17:59,Tarde,Enero,Domingo,Guatemala,Palencia,Ignorada,Hombre,17.0,15-19,15-19,15 - 19,Menor,Motocicleta,Freedom,Negro,2017,2010-2019,Colisión,Lesionado,Internado
2,2023,1,15.0,12:00 a 17:59,Tarde,Enero,Domingo,Guatemala,Guatemala,24,Hombre,22.0,20-24,20-24,20 - 24,Mayor,Motocicleta,Bajaj,Azul,2022,2020-2029,Colisión,Fallecido,No internado


In [0]:
display(fallecidos_df.describe())

summary,núm_corre,año_ocu,día_ocu,hora_ocu,g_hora,g_hora_5,mes_ocu,día_sem_ocu,depto_ocu,mupio_ocu,zona_ocu,sexo_per,edad_per,g_edad_80ymás,g_edad_60ymás,edad_quinquenales,mayor_menor,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,tipo_eve,fall_les,int_o_noint
count,11198,11198,11198,11175,11198,11198,11198,11198,11198,11198,11198,11198,9890,11198,11198,11198,11198,11198,11198,11198,11198,11198,11198,11198,11198
mean,5599.5,2023.0,15.565726022504018,13.133064876957494,null,null,null,null,null,null,7.817241379310345,null,31.42386248736097,null,null,null,null,null,null,null,2013.459808881394,null,null,null,null
stddev,3232.72849153776,0.0,8.894081296880344,7.0078910017282325,null,null,null,null,null,null,5.49278356742296,null,15.846088170578124,null,null,null,null,null,null,null,10.069058044545425,null,null,null,null
min,1,2023,1,0.0,00:00 a 05:59,Ignorada,Abril,Domingo,Alta Verapaz,Acatenango,0,Hombre,0.0,15-19,15-19,0 - 4,Ignorado,Automóvil,AHM,Amarillo,1974,1970-1979,Atropello,Fallecido,Ignorado
max,11198,2023,31,23.0,Ignorada,Tarde,Septiembre,Viernes,Zacapa,Zunilito,Ignorada,Mujer,98.0,Menor de 15,Menor de 15,Ignorado,Menor,Tractor,Zongshen,Verde,Ignorado,Ignorado,Vuelco,Lesionado,No internado


**Hechos de transito**

In [0]:
display(hechos_df.limit(2))

núm_corre,año_ocu,día_ocu,hora_ocu,g_hora,g_hora_5,mes_ocu,día_sem_ocu,depto_ocu,mupio_ocu,zona_ocu,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,tipo_eve
1,2023,1,15.0,12:00 a 17:59,Tarde,Enero,Domingo,Guatemala,Palencia,Ignorada,Pick up,Nissan,Negro,2017,2010-2019,Colisión
2,2023,1,15.0,12:00 a 17:59,Tarde,Enero,Domingo,Guatemala,Guatemala,18,Pick up,Nissan,Gris,1986,1980-1989,Colisión


In [0]:
display(hechos_df.describe())

summary,núm_corre,año_ocu,día_ocu,hora_ocu,g_hora,g_hora_5,mes_ocu,día_sem_ocu,depto_ocu,mupio_ocu,zona_ocu,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,tipo_eve
count,8218,8218,8218,8202,8218,8218,8218,8218,8218,8218,8218,8218,8218,8218,8218,8218,8218
mean,4109.5,2023.0,15.457897298612801,13.260424286759328,null,null,null,null,null,null,7.776038121170864,null,null,null,2010.6240900066182,null,null
stddev,2372.47658927684,0.0,8.863451945494145,7.16667739848595,null,null,null,null,null,null,5.386417585977838,null,null,null,10.245528751608635,null,null
min,1,2023,1,0.0,00:00 a 05:59,Ignorada,Abril,Domingo,Alta Verapaz,Acatenango,1,Automóvil,AHM,Amarillo,1973,1970-1979,Atropello
max,8218,2023,31,23.0,Ignorada,Tarde,Septiembre,Viernes,Zacapa,Zunilito,Ignorada,Tractor,Zongshen,Verde,Ignorado,Ignorado,Vuelco


**Vehiculos**

In [0]:
display(vehiculos_df.limit(2))

núm_corre,año_ocu,día_ocu,hora_ocu,g_hora,g_hora_5,mes_ocu,día_sem_ocu,depto_ocu,mupio_ocu,zona_ocu,sexo_per,edad_per,g_edad_80ymás,g_edad_60ymás,edad_quinquenales,estado_con,mayor_menor,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,tipo_eve
1,2023,1,15.0,12:00 a 17:59,Tarde,Enero,Domingo,Guatemala,Palencia,Ignorado,Hombre,38.0,35-39,35-39,35 - 39,Ignorado,Mayor,Pick up,Nissan,Negro,2017,2010-2019,Colisión
2,2023,1,15.0,12:00 a 17:59,Tarde,Enero,Domingo,Guatemala,Palencia,Ignorado,Hombre,17.0,15-19,15-19,15 - 19,Ignorado,Menor,Motocicleta,Freedom,Negro,2017,2010-2019,Colisión


In [0]:
display(vehiculos_df.describe())

summary,núm_corre,año_ocu,día_ocu,hora_ocu,g_hora,g_hora_5,mes_ocu,día_sem_ocu,depto_ocu,mupio_ocu,zona_ocu,sexo_per,edad_per,g_edad_80ymás,g_edad_60ymás,edad_quinquenales,estado_con,mayor_menor,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,tipo_eve
count,12197,12197,12197,12178,12197,12197,12197,12197,12197,12197,12197,12197,9389,12197,12197,12197,12197,12197,12197,12197,12197,12197,12197,12197
mean,6099.0,2023.0,15.520209887677298,13.357283626211201,null,null,null,null,null,null,7.6629771854529025,null,33.16519331132176,null,null,null,null,null,null,null,null,2011.8687022900763,null,null
stddev,3521.1149512618867,0.0,8.902805360201025,7.0593307885329635,null,null,null,null,null,null,5.40441239037978,null,12.557845640465363,null,null,null,null,null,null,null,null,10.077805504296196,null,null
min,1,2023,1,0.0,00:00 a 05:59,Ignorada,Abril,Domingo,Alta Verapaz,Acatenango,1,Hombre,9.0,15-19,15-19,10 - 14,Ebrio,Ignorado,Automóvil,AHM,Amarillo,1973,1970-1979,Atropello
max,12197,2023,31,23.0,Ignorada,Tarde,Septiembre,Viernes,Zacapa,Zunilito,Ignorado,Mujer,88.0,Menor de 15,Menor de 15,Ignorado,No Ebrio,Menor,Tractor,Zongshen,Verde,Ignorado,Ignorado,Vuelco


Además es importante notar que todos los datos son exclusivamente de los años 2023 sin importar el dataset.

In [0]:
fallecidos_df.select("año_ocu").distinct().show()
hechos_df.select("año_ocu").distinct().show()
vehiculos_df.select("año_ocu").distinct().show()

+-------+
|año_ocu|
+-------+
|   2023|
+-------+

+-------+
|año_ocu|
+-------+
|   2023|
+-------+

+-------+
|año_ocu|
+-------+
|   2023|
+-------+



Enfocandonos en el dataset de fallecidos, se encontraron 8 posibles causas de muerte que se ven listadas a continuación:

In [0]:
display(fallecidos_df.select("tipo_eve").distinct())

tipo_eve
Embarrancó
Vuelco
Choque
Derrape
Atropello
Colisión
Caída
Ignorado


En cuando a la disposicion geografica de los accidentes, se descubrio que el departamento "Sacatepéques" se encontraba mal escrito en los datasets, asi que fue corrigido antes de hacer el conteo. Con ello se observa que hay registros a lo largo de los 22 departamentos.

In [0]:
fallecidos_df = fallecidos_df.withColumn(
    "depto_ocu",
    f.when(f.col("depto_ocu") == "Sacatepequez", "Sacatepéquez").otherwise(f.col("depto_ocu"))
)
hechos_df = hechos_df.withColumn(
    "depto_ocu",
    f.when(f.col("depto_ocu") == "Sacatepequez", "Sacatepéquez").otherwise(f.col("depto_ocu"))
)
vehiculos_df = vehiculos_df.withColumn(
    "depto_ocu",
    f.when(f.col("depto_ocu") == "Sacatepequez", "Sacatepéquez").otherwise(f.col("depto_ocu"))
)

unique_deptos = (
    fallecidos_df.select("depto_ocu").distinct()
    .union(hechos_df.select("depto_ocu").distinct())
    .union(vehiculos_df.select("depto_ocu").distinct())
    .distinct()
)



display(unique_deptos.limit(25))

depto_ocu
Quiché
Alta Verapaz
San Marcos
Guatemala
Zacapa
Suchitepéquez
Chiquimula
El Progreso
Huehuetenango
Escuintla


## Análisis de Datos

**Enfocandonos en los hechos de tráfico**, y como es de esperar gracias a la gran densidad vehicular, Guatemala es el departamento con mayor cantidad de hechos registrados seguidos de Escuintla, esto puede ser causado por una mayor presencia de cuerpos de la PNC en este departamento así como mayor cantidad de vehículos.

In [0]:
# Agrupar por año y departamento, contar accidentes
accidents_by_year_dept = hechos_df.groupBy("año_ocu", "depto_ocu") \
    .agg(f.count("*").alias("total_accidents")) \
    .orderBy("año_ocu", "depto_ocu")

display(accidents_by_year_dept)

año_ocu,depto_ocu,total_accidents
2023,Alta Verapaz,356
2023,Baja Verapaz,145
2023,Chimaltenango,300
2023,Chiquimula,112
2023,El Progreso,173
2023,Escuintla,930
2023,Guatemala,3457
2023,Huehuetenango,126
2023,Izabal,244
2023,Jalapa,118


Databricks visualization. Run in Databricks to view.

En el eje temporal, la mayor cantidad de accidentes ocurre los fines de semanas y viernes. Algo esperable considerando que son los dias dedicados al ocio y el consumo de alcohol es más común.

In [0]:
accidents_by_weekday_2023 = hechos_df.groupBy("día_sem_ocu") \
    .agg(f.count("*").alias("accidentes_totales")) \
    .orderBy(f.col("accidentes_totales").desc())

display(accidents_by_weekday_2023)

día_sem_ocu,accidentes_totales
Domingo,1620
Sábado,1557
Viernes,1175
Lunes,1053
Jueves,956
Martes,933
Miércoles,924


Databricks visualization. Run in Databricks to view.

En cuanto a cuando suceden esos accidentes en el dia, es las noches y madrugadas del dia siguiete, aunque tambien hay un repunte importante a medio día cuando que coincide con las hora de almuerzo y salir a comer a algún lado.

In [0]:
accidents_by_hour_guatemala = hechos_df.filter(f.col("mupio_ocu") == "Guatemala") \
    .orderBy("hora_ocu")

display(accidents_by_hour_guatemala)

núm_corre,año_ocu,día_ocu,hora_ocu,g_hora,g_hora_5,mes_ocu,día_sem_ocu,depto_ocu,mupio_ocu,zona_ocu,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,tipo_eve
5523,2023,28,null,Ignorada,Ignorada,Agosto,Lunes,Guatemala,Guatemala,12,Automóvil,Ignorado,Ignorado,Ignorado,Ignorado,Atropello
6557,2023,25,null,Ignorada,Ignorada,Octubre,Miércoles,Guatemala,Guatemala,7,Motocicleta,Ignorado,Ignorado,Ignorado,Ignorado,Colisión
6625,2023,5,null,Ignorada,Ignorada,Octubre,Jueves,Guatemala,Guatemala,7,Automóvil,Ignorado,Ignorado,Ignorado,Ignorado,Atropello
7700,2023,18,null,Ignorada,Ignorada,Diciembre,Lunes,Guatemala,Guatemala,12,Motocicleta,Ignorado,Negro,Ignorado,Ignorado,Derrape
44,2023,3,0.0,00:00 a 05:59,Mañana,Enero,Martes,Guatemala,Guatemala,10,Automóvil,Volkswagen,Blanco,Ignorado,Ignorado,Colisión
232,2023,14,0.0,00:00 a 05:59,Mañana,Enero,Sábado,Guatemala,Guatemala,17,Camión,Hino,Blanco,2022,2020-2029,Colisión
235,2023,14,0.0,00:00 a 05:59,Mañana,Enero,Sábado,Guatemala,Guatemala,7,Motocicleta,Ignorado,Azul,Ignorado,Ignorado,Colisión
324,2023,20,0.0,00:00 a 05:59,Mañana,Enero,Viernes,Guatemala,Guatemala,13,Motocicleta,Ignorado,Ignorado,Ignorado,Ignorado,Derrape
341,2023,21,0.0,00:00 a 05:59,Mañana,Enero,Sábado,Guatemala,Guatemala,7,Automóvil,Toyota,Gris,2015,2010-2019,Colisión
343,2023,21,0.0,00:00 a 05:59,Mañana,Enero,Sábado,Guatemala,Guatemala,7,Motocicleta,Ignorado,Negro,Ignorado,Ignorado,Derrape


Databricks visualization. Run in Databricks to view.

Si juntamos los hechos de trafico con los datos de vehículos, utilizando valores temporales y geográficos en común logramos tener 4841 coincidencias, que es poco menos de la mitad de carros, lo que significa que al menos en cada accidente participan 2 carros, lo que hace sentido considereando como funcionan los accidentes, al tener una victima un culpable.

In [0]:
df1_alias = hechos_df.alias("df1")
df2_alias = vehiculos_df.alias("df2")
hechos_vehiculos_df = df1_alias.join(
    df2_alias,
    on=["día_ocu", "hora_ocu", "tipo_eve", "depto_ocu", "mupio_ocu", "zona_ocu"],
    how="inner"
)
hechos_vehiculos_df.count()

4841

In [0]:
vehiculos_per_accident = hechos_vehiculos_df.groupBy("df1.núm_corre", "df1.depto_ocu") \
    .agg(f.count("*").alias("num_vehiculos")) \
    .orderBy(f.col("num_vehiculos").desc())

display(vehiculos_per_accident)

núm_corre,depto_ocu,num_vehiculos
526,Guatemala,8
5381,Guatemala,8
604,Guatemala,7
6218,Guatemala,7
735,Guatemala,6
1199,Guatemala,6
5720,Guatemala,6
7402,Guatemala,6
5557,Guatemala,6
3257,Guatemala,6


Si agrupamos los la cantidad de carros por accidente, se puede obtener la cantidad promedio de carros que participan en un accidente por departamente como se ve acontinuación, siendo en todos los departamentos aproximadamente 2 vehículos.

In [0]:
avg_vehicles_per_accident = (
    vehiculos_per_accident.groupBy("df1.depto_ocu")
    .agg(f.avg("num_vehiculos").alias("prom_vehiculos"))
)

display(avg_vehicles_per_accident)

depto_ocu,prom_vehiculos
Guatemala,1.6629834254143647
Baja Verapaz,1.625
Chimaltenango,1.7307692307692308
Retalhuleu,1.5806451612903225
Quetzaltenango,1.7164179104477613
Huehuetenango,1.8541666666666667
San Marcos,1.7272727272727273
Alta Verapaz,1.4901960784313726
Escuintla,1.5573770491803278
Quiché,1.5555555555555556


A continuación un gráfico de los departamentos con mayor cantidad de carros por accidente en promedio:

In [0]:
avg_vehicles_per_accident.write.mode("overwrite").parquet("/Volumes/workspace/default/lab8/avg_vehicles_per_accident")
avg_df = spark.read.parquet("/Volumes/workspace/default/lab8/avg_vehicles_per_accident")
top10 = avg_df.orderBy(f.col("prom_vehiculos").desc()).limit(10)
display(top10)

depto_ocu,prom_vehiculos
Chiquimula,1.8571428571428572
Huehuetenango,1.8541666666666667
Sololá,1.75
Chimaltenango,1.7307692307692308
San Marcos,1.7272727272727273
Quetzaltenango,1.7164179104477613
Jutiapa,1.7142857142857142
Jalapa,1.7142857142857142
Zacapa,1.6666666666666667
Guatemala,1.6629834254143647


Databricks visualization. Run in Databricks to view.